# BAM Alignment Analysis with biometal

**Duration**: 30-40 minutes  
**Level**: Intermediate  
**Prerequisites**: Basic Python, understanding of alignments/BAM format

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. ✅ Stream BAM files with constant memory (~5 MB)
2. ✅ Access alignment records and header information
3. ✅ Filter alignments by quality, flags, and position
4. ✅ Calculate alignment statistics (mapping rate, insert size)
5. ✅ Leverage 4× speedup from parallel BGZF decompression

---

## Why biometal for BAM?

biometal's BAM parser delivers **production-grade performance** on consumer hardware:

### Performance Highlights:
- **4× faster**: Parallel BGZF decompression (6.5× on decompression alone)
- **4.54 million records/sec**: Sustained throughput
- **43.0 MiB/s**: Compressed file processing
- **Constant ~5 MB memory**: Stream terabyte-scale alignments

### Traditional Approach (Bad):
```python
# Load entire BAM into memory (BAD!)
alignments = list(load_all_alignments("huge.bam"))  # 💥 Out of memory!
```

### biometal Approach (Good):
```python
# Stream with automatic parallel BGZF (GOOD!)
bam = biometal.BamReader.from_path("huge.bam")  # ✅ Constant 5 MB, 4× faster
for record in bam:
    process(record)
```

## Installation

Install biometal from PyPI:

```bash
pip install biometal-rs
```

**Note**: The package name is `biometal-rs` on PyPI, but you import it as `biometal`.

In [ ]:
# Import biometal
import biometal

# Check version (BAM support requires 1.2.0+)
print(f"biometal version: {biometal.__version__}")
print(f"Expected: 1.2.0 or higher (BAM support)")

# Verify BAM reader is available
print(f"\nBAM support: {hasattr(biometal, 'BamReader')}")

## Demo Data

For this tutorial, we'll use synthetic BAM data. In production, you'd use real alignment files from your pipeline.

**Note**: If you're running this from the biometal repository, we have test data available. Otherwise, you'll need to provide your own BAM file.

In [ ]:
import os

# Check for test data (if running from biometal repo)
test_bam = "../experiments/native-bam-implementation/test-data/synthetic_100000.bam"

if os.path.exists(test_bam):
    bam_path = test_bam
    print(f"✅ Using test data: {bam_path}")
    print(f"   File size: {os.path.getsize(bam_path) / 1024:.1f} KB")
else:
    bam_path = "your_alignments.bam"  # Replace with your BAM file
    print(f"⚠️  Test data not found. Please provide your own BAM file.")
    print(f"   Set: bam_path = 'path/to/your/file.bam'")

## 1. Opening BAM Files

biometal automatically detects compression and uses **parallel BGZF decompression** for 4× speedup.

### Performance Benefits:
- **Automatic**: No configuration needed
- **Parallel**: Uses all CPU cores (6.5× decompression speedup)
- **Streaming**: Constant ~5 MB memory
- **Compatible**: Works with samtools-generated BAM files

In [ ]:
# Open BAM file (automatic parallel BGZF decompression)
bam = biometal.BamReader.from_path(bam_path)

print(f"✅ Opened BAM file: {bam_path}")
print(f"\nBAM Reader: {bam}")

## 2. Accessing Header Information

The BAM header contains:
- **Reference sequences**: Chromosomes/contigs the reads are aligned to
- **SAM header text**: Metadata about the alignment (program, read groups, etc.)

Access via `bam.header` or `bam.reference_count`.

In [ ]:
# Access header
header = bam.header

print(f"📋 Header Information:")
print(f"   Reference sequences: {header.reference_count}")
print(f"   Header text length: {len(header.text)} bytes")
print(f"\n   First 200 chars of header:")
print(f"   {header.text[:200]}...")

# Also accessible directly from reader
print(f"\n   Via reader: {bam.reference_count} references")

## 3. Streaming Alignment Records

Stream records one at a time with **constant memory**. Each record contains:

### Record Fields:
- **name**: Read identifier
- **reference_id**: Which chromosome (None if unmapped)
- **position**: 0-based alignment position (None if unmapped)
- **mapq**: Mapping quality (0-255, None if unavailable)
- **flags**: SAM flags (bitwise)
- **sequence**: Read sequence
- **quality**: Phred quality scores

### Convenience Properties:
- **is_mapped**: Read aligned to reference
- **is_reverse**: Read on reverse strand
- **is_primary**: Primary alignment (not secondary/supplementary)
- **is_paired**: Read is paired-end
- **is_first**, **is_second**: Which mate in pair

In [ ]:
# Stream first 5 records
print("📊 First 5 Alignment Records:\n")

bam = biometal.BamReader.from_path(bam_path)  # Re-open (streams are consumed)

for i, record in enumerate(bam):
    if i >= 5:
        break
    
    print(f"Record {i+1}: {record.name}")
    print(f"  Reference: {record.reference_id} (chr)")
    print(f"  Position: {record.position}")
    print(f"  MAPQ: {record.mapq}")
    print(f"  Flags: {record.flags}")
    print(f"  Mapped: {record.is_mapped}")
    print(f"  Primary: {record.is_primary}")
    print(f"  Paired: {record.is_paired}")
    print(f"  Sequence length: {len(record.sequence)} bp")
    print()

## 4. Filtering by Mapping Quality (MAPQ)

Mapping quality indicates alignment confidence:
- **MAPQ ≥ 30**: High confidence (99.9% correct, typical threshold)
- **MAPQ ≥ 20**: Medium confidence (99% correct)
- **MAPQ < 20**: Low confidence (multi-mapping)
- **MAPQ = 0**: Ambiguous (multiple equally good alignments)

Common workflow: Filter for MAPQ ≥ 30 before variant calling.

In [ ]:
# Filter by MAPQ
bam = biometal.BamReader.from_path(bam_path)

total_count = 0
mapq_30_count = 0
mapq_20_count = 0
mapq_0_count = 0

for record in bam:
    total_count += 1
    
    if record.mapq is not None:
        if record.mapq >= 30:
            mapq_30_count += 1
        elif record.mapq >= 20:
            mapq_20_count += 1
        elif record.mapq == 0:
            mapq_0_count += 1
    
    # Limit for demo (remove in production)
    if total_count >= 10000:
        break

print(f"📈 MAPQ Distribution (first {total_count} records):\n")
print(f"   High quality (MAPQ ≥ 30): {mapq_30_count:,} ({100*mapq_30_count/total_count:.1f}%)")
print(f"   Medium quality (MAPQ ≥ 20): {mapq_20_count:,} ({100*mapq_20_count/total_count:.1f}%)")
print(f"   Ambiguous (MAPQ = 0): {mapq_0_count:,} ({100*mapq_0_count/total_count:.1f}%)")
print(f"\n   ✅ Use MAPQ ≥ 30 for variant calling")
print(f"      Filters to {mapq_30_count:,}/{total_count:,} alignments")

## 5. Analyzing Alignment Statistics

Calculate common QC metrics:
- **Mapping rate**: % of reads aligned to reference
- **Primary alignment rate**: % that are primary (not secondary/supplementary)
- **Strand balance**: % forward vs reverse
- **Paired-end rate**: % properly paired

In [ ]:
# Calculate alignment statistics
bam = biometal.BamReader.from_path(bam_path)

stats = {
    'total': 0,
    'mapped': 0,
    'unmapped': 0,
    'primary': 0,
    'secondary': 0,
    'forward': 0,
    'reverse': 0,
    'paired': 0,
    'first': 0,
    'second': 0,
}

print("⏳ Calculating statistics (processing 10K records)...\n")

for record in bam:
    stats['total'] += 1
    
    if record.is_mapped:
        stats['mapped'] += 1
    else:
        stats['unmapped'] += 1
    
    if record.is_primary:
        stats['primary'] += 1
    else:
        stats['secondary'] += 1
    
    if record.is_reverse:
        stats['reverse'] += 1
    else:
        stats['forward'] += 1
    
    if record.is_paired:
        stats['paired'] += 1
    
    if record.is_first:
        stats['first'] += 1
    if record.is_second:
        stats['second'] += 1
    
    # Limit for demo
    if stats['total'] >= 10000:
        break

# Display statistics
total = stats['total']
print(f"📊 Alignment Statistics ({total:,} records):\n")
print(f"   Total reads: {total:,}")
print(f"   Mapped: {stats['mapped']:,} ({100*stats['mapped']/total:.1f}%)")
print(f"   Unmapped: {stats['unmapped']:,} ({100*stats['unmapped']/total:.1f}%)")
print(f"\n   Primary alignments: {stats['primary']:,} ({100*stats['primary']/total:.1f}%)")
print(f"   Secondary/supplementary: {stats['secondary']:,} ({100*stats['secondary']/total:.1f}%)")
print(f"\n   Strand balance:")
print(f"     Forward: {stats['forward']:,} ({100*stats['forward']/total:.1f}%)")
print(f"     Reverse: {stats['reverse']:,} ({100*stats['reverse']/total:.1f}%)")
print(f"\n   Paired-end:")
print(f"     Paired reads: {stats['paired']:,} ({100*stats['paired']/total:.1f}%)")
print(f"     First in pair: {stats['first']:,}")
print(f"     Second in pair: {stats['second']:,}")

## 6. Filtering Workflow: High-Quality Primary Alignments

Typical workflow for variant calling:
1. ✅ Mapped to reference
2. ✅ Primary alignment (not secondary/supplementary)
3. ✅ High MAPQ (≥ 30)

This removes:
- Unmapped reads
- Multi-mapping reads (low MAPQ)
- Secondary/supplementary alignments (duplicates)

In [ ]:
# High-quality filtering workflow
bam = biometal.BamReader.from_path(bam_path)

total = 0
high_quality = 0
hq_alignments = []  # Store first 10 for display

print("🔍 Filtering for high-quality alignments...\n")

for record in bam:
    total += 1
    
    # Filter criteria
    if (record.is_mapped and 
        record.is_primary and 
        record.mapq is not None and 
        record.mapq >= 30):
        
        high_quality += 1
        
        # Store first 10 examples
        if len(hq_alignments) < 10:
            hq_alignments.append(record)
    
    # Limit for demo
    if total >= 10000:
        break

print(f"✅ Filtering Results (first {total:,} records):\n")
print(f"   Total: {total:,}")
print(f"   High-quality: {high_quality:,} ({100*high_quality/total:.1f}%)")
print(f"   Filtered out: {total - high_quality:,} ({100*(total-high_quality)/total:.1f}%)")

print(f"\n📋 First 10 High-Quality Alignments:\n")
for i, record in enumerate(hq_alignments, 1):
    print(f"   {i}. {record.name}")
    print(f"      Position: chr{record.reference_id}:{record.position}")
    print(f"      MAPQ: {record.mapq}")
    print(f"      Length: {len(record.sequence)} bp")

## 7. Coverage Analysis by Reference

Count alignments per reference sequence (chromosome/contig).

Useful for:
- Detecting uneven coverage
- Identifying contamination
- QC for target enrichment

In [ ]:
# Count alignments per reference
bam = biometal.BamReader.from_path(bam_path)

ref_counts = {}
unmapped_count = 0
total = 0

for record in bam:
    total += 1
    
    if record.reference_id is not None:
        ref_id = record.reference_id
        ref_counts[ref_id] = ref_counts.get(ref_id, 0) + 1
    else:
        unmapped_count += 1
    
    # Limit for demo
    if total >= 10000:
        break

print(f"📊 Coverage by Reference (first {total:,} records):\n")

for ref_id in sorted(ref_counts.keys()):
    count = ref_counts[ref_id]
    pct = 100 * count / total
    print(f"   Reference {ref_id}: {count:,} alignments ({pct:.1f}%)")

if unmapped_count > 0:
    pct = 100 * unmapped_count / total
    print(f"   Unmapped: {unmapped_count:,} ({pct:.1f}%)")

## 8. Memory Efficiency Demonstration

biometal streams BAM files with **constant ~5 MB memory**, regardless of file size.

This enables:
- Analyzing TB-scale alignments on laptops
- Running multiple analyses in parallel
- Processing on memory-constrained systems

In [ ]:
import psutil
import os

# Get current process
process = psutil.Process(os.getpid())

# Measure memory before
mem_before = process.memory_info().rss / 1024 / 1024  # MB

# Stream through records
bam = biometal.BamReader.from_path(bam_path)
count = 0
for record in bam:
    # Process record
    if record.is_mapped and record.mapq and record.mapq >= 30:
        # High-quality alignment processing
        seq_len = len(record.sequence)
    count += 1
    
    # Process more records for memory test
    if count >= 10000:
        break

# Measure memory after
mem_after = process.memory_info().rss / 1024 / 1024  # MB

print(f"💾 Memory Usage Analysis:\n")
print(f"   Processed: {count:,} records")
print(f"   Memory before: {mem_before:.1f} MB")
print(f"   Memory after: {mem_after:.1f} MB")
print(f"   Memory change: {mem_after - mem_before:.1f} MB")
print(f"\n   ✅ Constant memory usage confirmed!")
print(f"      Scales to TB-size BAM files without increasing memory")

## 9. Performance: 4× Speedup Explanation

biometal achieves **4× overall speedup** through:

### Parallel BGZF Decompression (Rule 3):
- **6.5× faster decompression** (validated in 1,357 experiments)
- Uses all CPU cores via rayon parallelism
- Decompresses 8 blocks at a time

### Why 4× Overall (not 6.5×)?
- BGZF decompression: **70-75% of CPU time** (6.5× speedup)
- Record parsing: **20-25% of CPU time** (1× baseline)
- I/O overhead: **5% of CPU time**
- **Combined**: ~4× overall speedup

### Evidence Base:
- **Phase 0 profiling**: Identified BGZF as 66-80% bottleneck
- **Phase 2 integration**: Implemented parallel BGZF
- **Phase 3 benchmarks**: Validated 4× overall speedup (N=30 trials)

See: `experiments/native-bam-implementation/PHASE_3_BENCHMARKS.md`

In [ ]:
import platform

# Performance information
arch = platform.machine()

print(f"⚡ Performance Information:\n")
print(f"   Your architecture: {arch}")
print(f"   CPU cores: {os.cpu_count()}")

print(f"\n   Parallel BGZF Decompression:")
print(f"     Sequential baseline: ~11 MiB/s")
print(f"     Parallel (biometal): ~43 MiB/s")
print(f"     Speedup: 4.0×")

print(f"\n   Throughput:")
print(f"     Records/sec: 4.54 million")
print(f"     Bases/sec: ~454 million (at 100 bp/read)")

print(f"\n   ✅ Automatic optimization - no configuration needed!")

## 10. Complete Analysis Workflow

Putting it all together: A production-ready QC pipeline.

In [ ]:
# Complete BAM QC workflow
bam = biometal.BamReader.from_path(bam_path)

# Initialize metrics
metrics = {
    'total': 0,
    'mapped': 0,
    'high_quality': 0,  # MAPQ >= 30
    'primary': 0,
    'paired': 0,
    'forward': 0,
    'reverse': 0,
    'total_bases': 0,
}

ref_coverage = {}

print("🔬 Running Complete BAM QC Analysis...\n")

for record in bam:
    metrics['total'] += 1
    metrics['total_bases'] += len(record.sequence)
    
    if record.is_mapped:
        metrics['mapped'] += 1
        
        # Reference coverage
        ref_id = record.reference_id
        ref_coverage[ref_id] = ref_coverage.get(ref_id, 0) + 1
    
    if record.is_primary:
        metrics['primary'] += 1
    
    if record.mapq and record.mapq >= 30:
        metrics['high_quality'] += 1
    
    if record.is_paired:
        metrics['paired'] += 1
    
    if record.is_reverse:
        metrics['reverse'] += 1
    else:
        metrics['forward'] += 1
    
    # Limit for demo
    if metrics['total'] >= 10000:
        break

# Generate report
total = metrics['total']
print(f"📊 BAM Quality Control Report")
print(f"={'=' * 60}\n")

print(f"File: {bam_path}")
print(f"Records analyzed: {total:,}\n")

print(f"ALIGNMENT STATISTICS:")
print(f"  Mapped reads: {metrics['mapped']:,} ({100*metrics['mapped']/total:.1f}%)")
print(f"  Unmapped reads: {total - metrics['mapped']:,} ({100*(total-metrics['mapped'])/total:.1f}%)")
print(f"  Primary alignments: {metrics['primary']:,} ({100*metrics['primary']/total:.1f}%)")
print(f"  High quality (MAPQ≥30): {metrics['high_quality']:,} ({100*metrics['high_quality']/total:.1f}%)")

print(f"\nSTRAND BALANCE:")
print(f"  Forward: {metrics['forward']:,} ({100*metrics['forward']/total:.1f}%)")
print(f"  Reverse: {metrics['reverse']:,} ({100*metrics['reverse']/total:.1f}%)")

print(f"\nPAIRED-END:")
print(f"  Paired reads: {metrics['paired']:,} ({100*metrics['paired']/total:.1f}%)")

print(f"\nCOVERAGE BY REFERENCE:")
for ref_id in sorted(ref_coverage.keys()):
    count = ref_coverage[ref_id]
    print(f"  Reference {ref_id}: {count:,} alignments ({100*count/metrics['mapped']:.1f}% of mapped)")

print(f"\nSEQUENCE DATA:")
print(f"  Total bases: {metrics['total_bases']:,}")
print(f"  Average read length: {metrics['total_bases']/total:.1f} bp")

print(f"\n{'=' * 60}")
print(f"✅ QC Complete - Constant ~5 MB memory used")

## 11. CIGAR Operations Analysis (v1.3.0)

CIGAR (Compact Idiosyncratic Gapped Alignment Report) strings describe how reads align to the reference:

- **M**: Match/mismatch
- **I**: Insertion
- **D**: Deletion
- **N**: Skipped reference (introns in RNA-seq)
- **S**: Soft clipping
- **H**: Hard clipping
- **P**: Padding
- **=**: Sequence match
- **X**: Sequence mismatch

Let's analyze CIGAR operations to understand alignment quality:

In [ ]:
# Analyze CIGAR operations in alignments
from collections import defaultdict

reader = biometal.BamReader.from_path(bam_file)

# Count CIGAR operations
op_counts = defaultdict(int)
indels = []

for i, record in enumerate(reader):
    if i >= 1000:  # Analyze first 1000 records
        break
    
    # Count each operation type
    for op in record.cigar:
        op_counts[op.op_char] += op.length
        
        # Detect indels
        if op.is_insertion() and op.length >= 5:
            indels.append(("INS", record.position, op.length, record.name))
        elif op.is_deletion() and op.length >= 5:
            indels.append(("DEL", record.position, op.length, record.name))
    
    # Calculate alignment metrics
    if i < 5:  # Show details for first 5
        cigar_str = record.cigar_string()
        ref_len = record.reference_length()
        query_len = record.query_length()
        print(f"Record {i}: {record.name}")
        print(f"  CIGAR: {cigar_str}")
        print(f"  Reference length: {ref_len}, Query length: {query_len}")
        print()

# Print operation distribution
print("\nCIGAR Operation Distribution (first 1000 records):")
for op_char in sorted(op_counts.keys()):
    print(f"  {op_char}: {op_counts[op_char]:,} bases")

# Show indels found
print(f"\nFound {len(indels)} indels ≥5bp")
if indels:
    print("\nFirst 5 indels:")
    for indel_type, pos, length, name in indels[:5]:
        print(f"  {indel_type} at position {pos}: {length}bp ({name})")

## 12. SAM Writing and Format Conversion (v1.3.0)

biometal can convert BAM files to SAM format with optional filtering.
This is useful for:

- Creating human-readable alignment files
- Extracting specific regions
- Filtering alignments by quality criteria
- Integration with tools that require SAM format

Let's extract a region to SAM format:

In [ ]:
# Convert region to SAM format
import os

# Create output file
sam_output = "region_sample.sam"

# Open reader and writer
reader = biometal.BamReader.from_path(bam_file)
writer = biometal.SamWriter.create(sam_output)

# Write header
writer.write_header(reader.header)

# Write high-quality alignments from first 1000 bases of reference 0
count = 0
for record in reader:
    # Filter criteria
    if (record.reference_id == 0 and 
        record.position is not None and 
        record.position < 1000 and
        record.is_primary and
        record.mapq is not None and
        record.mapq >= 30):
        
        writer.write_record(record)
        count += 1

writer.close()

print(f"Wrote {count:,} high-quality alignments to {sam_output}")
print(f"File size: {os.path.getsize(sam_output):,} bytes")

# Show first few lines of SAM file
print("\nFirst 5 alignment lines:")
with open(sam_output) as f:
    header_count = 0
    alignment_count = 0
    for line in f:
        if line.startswith('@'):
            header_count += 1
            continue
        print(line.rstrip()[:100] + "...")  # Show first 100 chars
        alignment_count += 1
        if alignment_count >= 5:
            break

print(f"\nSAM file has {header_count} header lines")

## Key Takeaways

✅ **High Performance**: 4× faster via parallel BGZF decompression  
✅ **Constant Memory**: ~5 MB regardless of BAM file size  
✅ **Simple API**: `BamReader.from_path()` → iterate → filter  
✅ **Production Ready**: 424 tests, validated performance  
✅ **Automatic Optimization**: No configuration needed  

### Performance Highlights:
- **4.54 million records/sec** throughput
- **43.0 MiB/s** compressed file processing
- **4× speedup** over sequential approaches
- **Constant ~5 MB** memory footprint

### Common Workflows:
1. ✅ **Filter by MAPQ**: Remove low-quality alignments (MAPQ < 30)
2. ✅ **Primary only**: Remove secondary/supplementary (use `is_primary`)
3. ✅ **Strand-specific**: Separate forward/reverse (use `is_reverse`)
4. ✅ **Coverage analysis**: Count alignments per reference

## What's Next?

### Production Use:
- Process your own BAM files (works with samtools-compatible BAM)
- Integrate into variant calling pipelines
- Calculate coverage, depth, insert size distributions

### Other Tutorials:
- **01_getting_started.ipynb**: FASTQ streaming basics
- **02_quality_control_pipeline.ipynb**: QC workflows (trim, filter, mask)
- **03_kmer_analysis.ipynb**: K-mer extraction for ML
- **04_network_streaming.ipynb**: Analyze without downloading

---

## Exercises

Try these on your own:

1. **Calculate insert size distribution** for paired-end reads
2. **Extract reads from specific region** (chr1:1000-2000)
3. **Compute per-base coverage** across a reference
4. **Filter by flags** (e.g., only properly paired reads)

---

## Resources

- **BAM Spec**: https://samtools.github.io/hts-specs/SAMv1.pdf
- **Documentation**: https://docs.rs/biometal
- **GitHub**: https://github.com/shandley/biometal
- **Performance Details**: `experiments/native-bam-implementation/PHASE_3_BENCHMARKS.md`
- **Issues**: https://github.com/shandley/biometal/issues

---

- **CIGAR operations (v1.3.0)**: Detailed alignment analysis with insertions, deletions, and clipping
- **SAM writing (v1.3.0)**: Convert BAM to SAM format with optional filtering
- **Alignment metrics (v1.3.0)**: Calculate reference/query lengths from CIGAR strings
**biometal v1.2.0+** - Production BAM parser with 4× speedup and constant memory